In [38]:
import numpy as np
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [39]:
training_file = 'simplified.txt'

In [40]:
raw_text = open(training_file, 'r', encoding='utf-8').read()

In [41]:
raw_text = raw_text.lower()

In [42]:
print(raw_text[:200])

in this scene, there was a council meeting organised by tywin lannister. in the scene, tywin was there to attend all the members, waiting at the head of the table, and there were 5 chairs placed at hi


In [43]:
nChars = len(raw_text)

In [44]:
chars = sorted(list(set(raw_text)))

In [45]:
nVocabs = len(chars)

In [46]:
print("The total number of vocabularies is: ", nVocabs)

The total number of vocabularies is:  31


In [47]:
allWords = raw_text.split()

In [48]:
uniqueWords = sorted(list(set(allWords)))

In [49]:
indexToChars = dict((i, c) for i, c in enumerate(chars))
charsToIndex = dict((c, i) for i, c in enumerate(chars))

In [50]:
import numpy as np

In [51]:
seqLength = 160

In [52]:
nSeq = int(np.floor((nChars - 1) / seqLength))

In [53]:
print("The total number of sequences is: ", nSeq)

The total number of sequences is:  5


In [54]:
X = np.zeros((nSeq, seqLength, nVocabs))
y = np.zeros((nSeq, seqLength, nVocabs))

In [55]:
for i in range(nSeq):

    xSequence = raw_text[i * seqLength: (i + 1) * seqLength]

    xSequenceIndex = [charsToIndex[char] for char in xSequence]

    inputSequence = np.zeros((seqLength, nVocabs))

    for j in range(seqLength):

        inputSequence[j][xSequenceIndex[j]] = 1

    X[i] = inputSequence

    ySequence = raw_text[i * seqLength + 1: (i + 1) * seqLength + 1]

    ySequenceIndex = [charsToIndex[char] for char in ySequence]

    targetSequence = np.zeros((seqLength, nVocabs))

    for j in range(seqLength):

        targetSequence[j][ySequenceIndex[j]] = 1

    y[i] = targetSequence

In [56]:
tensorflow.random.set_seed(42)

In [57]:
hiddenUnits = 70
dropOut = 0.4

In [58]:
batchSize = 100
nEpoch = 300

In [59]:
model = Sequential()

In [60]:
model.add(LSTM(hiddenUnits, input_shape=(None, nVocabs), return_sequences=True, dropout=dropOut))

In [61]:
model.add(LSTM(hiddenUnits, return_sequences=True, dropout=dropOut))

In [62]:
model.add(Dense(nVocabs, activation='softmax'))

In [63]:
optimizer = keras.optimizers.Adam(lr=0.001)

c:\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [64]:
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [65]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, None, 70)          28560     
                                                                 
 lstm_3 (LSTM)               (None, None, 70)          39480     
                                                                 
 dense_1 (Dense)             (None, None, 31)          2201      
                                                                 
Total params: 70,241
Trainable params: 70,241
Non-trainable params: 0
_________________________________________________________________
None


In [66]:
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping

In [67]:
filePath = 'model.hdf5'

In [68]:
checkpoint = ModelCheckpoint(filePath, monitor='loss', verbose=1, save_best_only=True, mode='min')

In [69]:
earlyStop = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=3)

In [70]:
import numpy as np

In [71]:
def generateText(model, genLength, nVocab, indexToChar):
    index = np.random.randint(nVocab)
    yChar = [indexToChar[index]]
    x = np.zeros((1, genLength, nVocab))

    for i in range(genLength):
        x[0, i, :][index] = 1
        pred = model.predict(x[:, :i+1, :])[0][-1]
        index = np.argmax(pred)
        yChar.append(indexToChar[index])

    return ''.join(yChar)

In [72]:
class ResultChecker(Callback):
    def __init__(self, model, genLength, nVocab, indexToChar):
        self.model = model
        self.genLength = genLength
        self.nVocab = nVocab
        self.indexToChar = indexToChar

    def on_epoch_end(self, epoch, logs={}):
        if epoch % 10 == 0:
            print(generateText(self.model, self.genLength, self.nVocab, self.indexToChar))
        # print(generateText(self.model, self.genLength, self.nVocab, self.indexToChar))

In [73]:
resultChecker = ResultChecker(model, 100, nVocabs, indexToChars)

In [74]:
model.fit(X, y, batch_size=batchSize, epochs=nEpoch, callbacks=[checkpoint, earlyStop, resultChecker])

Epoch 1/300


1/1 [==============================] - ETA: 0s - loss: 3.4363
Epoch 1: loss improved from inf to 3.43628, saving model to model.hdf5
1/1 [==============================] - 0s 22ms/step
uleeeccc..................l........l..........l.........l..........l.........l..........l.........l..
1/1 [==============================] - 8s 8s/step - loss: 3.4363
Epoch 2/300
1/1 [==============================] - ETA: 0s - loss: 3.4264
Epoch 2: loss improved from 3.43628 to 3.42640, saving model to model.hdf5
1/1 [==============================] - 0s 64ms/step - loss: 3.4264
Epoch 3/300
1/1 [==============================] - ETA: 0s - loss: 3.4159
Epoch 3: loss improved from 3.42640 to 3.41594, saving model to model.hdf5
1/1 [==============================] - 0s 62ms/step - loss: 3.4159
Epoch 4/300
1/1 [==============================] - ETA: 0s - loss: 3.4046
Epoch 4: loss improved from 3.41594 to 3.40463, saving model to model.hdf5
1/1 [==============================] - 0s 63ms/step - loss: 3.4046


In [75]:
# display the model results
print(generateText(model, 1000, nVocabs, indexToChars))

1/1 [==============================] - 0s 64ms/step
h                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   